In [ ]:
import tensorflow as tf

print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import zipfile
!curl -O https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

zip_ref=zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
import os

for dirpath, dirnames, filenames in os.walk("10_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

In [ ]:

# Setup data inputs
from keras_preprocessing.image import ImageDataGenerator

IMAGE_SHAPE = (224, 224)
BATCH_SIZE = 32

# train_dir = "/workspace/Artificial/TENSORFLOW/10_food_classes_10_percent/train"
# test_dir = "/workspace/Artificial/TENSORFLOW/10_food_classes_10_percent/test"

train_dir = '10_food_classes_10_percent/train'
test_dir = '10_food_classes_10_percent/test'
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

print("Training images:")
train_data_10_percent = train_datagen.flow_from_directory(train_dir,
                                               target_size=IMAGE_SHAPE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

print("Testing images:")
test_data = train_datagen.flow_from_directory(test_dir,
                                              target_size=IMAGE_SHAPE,
                                              batch_size=BATCH_SIZE,
                                              class_mode="categorical")

In [ ]:
from keras.api.callbacks import TensorBoard
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
    log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback=TensorBoard(log_dir=log_dir)
    print(f"Saving Tensorboard logs to {log_dir}")
    return tensorboard_callback

In [ ]:
import tensorflow as tf
from keras.api.layers import Dense,GlobalAveragePooling2D
from keras.api.applications import ResNet50V2
from keras.api.models import Sequential
from keras.api.callbacks import TensorBoard
import datetime

In [ ]:


# Define image shape
IMAGE_SHAPE = (224, 224)

# Load ResNet50V2 model without the top classification layer
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=IMAGE_SHAPE + (3,))
base_model.trainable = False  # Freeze the base model layers

# Create the new model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Reduce feature maps to a single vector
    Dense(10, activation='softmax', name='output_layer')  # Output layer with 10 classes
])

In [ ]:

# Compile the model
model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_keras", experiment_name="resnet50V2")],
    verbose=2
)


In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curves(history):

  loss = history.history['loss']
  val_loss = history.history['val_loss']

  accuracy = history.history['accuracy']
  val_accuracy = history.history['val_accuracy']

  epochs = range(len(history.history['loss']))

  # Plot loss
  plt.plot(epochs, loss, label='training_loss')
  plt.plot(epochs, val_loss, label='val_loss')
  plt.title('Loss')
  plt.xlabel('Epochs')
  plt.legend()

  # Plot accuracy
  plt.figure()
  plt.plot(epochs, accuracy, label='training_accuracy')
  plt.plot(epochs, val_accuracy, label='val_accuracy')
  plt.title('Accuracy')
  plt.xlabel('Epochs')
  plt.legend();
plot_loss_curves(history)

In [ ]:
model.summary()

In [ ]:
from keras.api.applications import EfficientNetV2B0

IMAGE_SHAPE = (224, 224)

base_model = EfficientNetV2B0(weights='imagenet', include_top=False, input_shape=IMAGE_SHAPE + (3,))
base_model.trainable = False

efficientnet_model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(10, activation='softmax', name='output_layer')
])

efficientnet_model.compile(
    loss='categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']
)

In [ ]:
efficientnet_history = efficientnet_model.fit(
    train_data_10_percent,
    epochs=5,
    steps_per_epoch=len(train_data_10_percent),
    validation_data=test_data,
    validation_steps=len(test_data),
    callbacks=[create_tensorboard_callback(dir_name="tensorflow_keras", experiment_name="efficientnetB0")],
    verbose=2
)